In [6]:
# ============================================================
# 3D "Google Earth-like" Satellite Globe + DBSCAN Clustering
# ============================================================
# - Ambil data orbit semua satelit aktif dari CelesTrak (JSON GP)
# - Konversi elemen orbit -> posisi 3D (ECI, aproksimasi)
# - Jalankan DBSCAN (implementasi manual, tanpa sklearn)
# - Visualisasi: Bumi 3D + satelit, warna berdasarkan cluster
#
# NOTE:
#   - Butuh internet (fetch JSON dari CelesTrak).
#   - Butuh plotly terinstal di environment ini.
# ============================================================

import math
import json
import urllib.request
import random

import numpy as np

try:
    import plotly.graph_objects as go
except ModuleNotFoundError:
    raise ModuleNotFoundError(
        "Module 'plotly' belum terinstall.\n"
        "Silakan gunakan virtualenv atau environment lain,\n"
        "lalu jalankan:  pip install plotly"
    )

# -----------------------------
# 1. Ambil data semua satelit aktif dari CelesTrak (JSON)
# -----------------------------
CELESTRAK_URL = (
    "https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=json"
)

print("Mengambil data satelit dari CelesTrak...")
with urllib.request.urlopen(CELESTRAK_URL, timeout=30) as resp:
    sats_raw = json.loads(resp.read().decode("utf-8"))

print(f"Jumlah entri satelit (raw): {len(sats_raw)}")

# -----------------------------
# 2. Fungsi: elemen orbit -> posisi 3D (ECI) approx
# -----------------------------
MU_EARTH = 398600.4418  # km^3/s^2
R_EARTH = 6378.137       # km

def kepler_to_cartesian(sat):
    """
    Konversi elemen orbit dari CelesTrak JSON ke posisi 3D ECI (x, y, z) dalam km.
    Ini bukan SGP4 penuh (jadi jangan dipakai buat navigasi presisi),
    tapi cukup untuk visualisasi distribusi orbit.
    """
    try:
        n_rev_day = float(sat["MEAN_MOTION"])        # rev / day
        e = float(sat["ECCENTRICITY"])
        inc_deg = float(sat["INCLINATION"])
        raan_deg = float(sat["RA_OF_ASC_NODE"])
        argp_deg = float(sat["ARG_OF_PERICENTER"])
        M_deg = float(sat["MEAN_ANOMALY"])
    except (KeyError, ValueError):
        return None

    # Mean motion -> rad/s
    n_rad_s = n_rev_day * 2.0 * math.pi / 86400.0

    # Semi-major axis a dari n
    a = (MU_EARTH / (n_rad_s ** 2)) ** (1.0 / 3.0)

    # Konversi derajat -> radian
    inc = math.radians(inc_deg)
    raan = math.radians(raan_deg)
    argp = math.radians(argp_deg)
    M = math.radians(M_deg)

    # Pecahkan persamaan Kepler: M = E - e sin E
    E = M
    for _ in range(10):
        E = E - (E - e * math.sin(E) - M) / (1 - e * math.cos(E))

    # True anomaly
    nu = 2.0 * math.atan2(
        math.sqrt(1 + e) * math.sin(E / 2.0),
        math.sqrt(1 - e) * math.cos(E / 2.0),
    )

    # Jarak dari pusat bumi (r)
    r = a * (1.0 - e * math.cos(E))

    # Posisi di bidang orbit (PQW frame)
    x_p = r * math.cos(nu)
    y_p = r * math.sin(nu)
    z_p = 0.0

    # Rotasi ke ECI: R3(raan) * R1(inc) * R3(argp) * [x_p, y_p, 0]
    cosO = math.cos(raan); sinO = math.sin(raan)
    cosi = math.cos(inc);  sini = math.sin(inc)
    cosw = math.cos(argp); sinw = math.sin(argp)

    # R3(argp)
    x1 = cosw * x_p - sinw * y_p
    y1 = sinw * x_p + cosw * y_p
    z1 = 0.0

    # R1(inc)
    x2 = x1
    y2 = cosi * y1 - sini * z1
    z2 = sini * y1 + cosi * z1

    # R3(raan)
    x = cosO * x2 - sinO * y2
    y = sinO * x2 + cosO * y2
    z = z2

    return x, y, z

# -----------------------------
# 3. Hitung posisi 3D untuk semua satelit
#    (dan batasi jumlah untuk DBSCAN supaya tidak terlalu berat)
# -----------------------------
positions = []
names = []

for sat in sats_raw:
    pos = kepler_to_cartesian(sat)
    if pos is None:
        continue
    positions.append(pos)
    names.append(sat.get("OBJECT_NAME", "").strip())

positions = np.array(positions)
print(f"Jumlah satelit dengan posisi valid: {positions.shape[0]}")

# Batas maksimum sampel untuk DBSCAN (biar tidak terlalu berat O(N^2))
MAX_SAMPLES = 3000
n_total = positions.shape[0]

if n_total > MAX_SAMPLES:
    idx = np.random.choice(n_total, size=MAX_SAMPLES, replace=False)
    X = positions[idx]
    names_sub = [names[i] for i in idx]
    print(f"DBSCAN hanya akan dijalankan pada subset {MAX_SAMPLES} satelit.")
else:
    X = positions
    names_sub = names

# -----------------------------
# 4. Implementasi DBSCAN sederhana (tanpa sklearn)
# -----------------------------
def dbscan(X, eps, min_pts):
    """
    Implementasi DBSCAN manual.
    X: array (N, d)
    eps: radius (dalam satuan yang sama dengan X, di sini km)
    min_pts: minimal tetangga untuk jadi core point
    Return: labels (N,) dengan nilai:
        -1 = noise
        0,1,2,... = ID cluster
    """
    N = X.shape[0]
    labels = np.full(N, -99, dtype=int)  # -99 = UNCLASSIFIED
    cluster_id = 0

    # fungsi regionQuery: cari indeks tetangga dalam radius eps
    def region_query(i):
        # hitung jarak dari titik i ke semua titik
        diff = X - X[i]
        dist2 = np.einsum("ij,ij->i", diff, diff)  # squared distance
        neighbors = np.where(dist2 <= eps**2)[0]
        return neighbors

    for i in range(N):
        if labels[i] != -99:
            continue  # sudah diklasifikasi

        neighbors = region_query(i)
        if neighbors.size < min_pts:
            labels[i] = -1  # noise
            continue

        # buat cluster baru
        cluster_id += 1
        labels[i] = cluster_id

        # queue untuk expand cluster
        seeds = list(neighbors)
        # pastikan titik i sudah included
        if i not in seeds:
            seeds.append(i)

        k = 0
        while k < len(seeds):
            j = seeds[k]
            if labels[j] == -1:
                labels[j] = cluster_id  # noise -> border point
            if labels[j] == -99:
                labels[j] = cluster_id
                new_neighbors = region_query(j)
                if new_neighbors.size >= min_pts:
                    # gabung neighbors baru
                    for n_idx in new_neighbors:
                        if n_idx not in seeds:
                            seeds.append(n_idx)
            k += 1

    # ganti semua -99 yang tersisa (kalau ada) jadi -1 (noise)
    labels[labels == -99] = -1
    return labels

# -----------------------------
# 5. Jalankan DBSCAN di posisi 3D
# -----------------------------
# eps ~ 1500 km; min_pts di-tuning sedikit
EPS_KM = 1500.0
MIN_PTS = 10

print("Menjalankan DBSCAN (ini O(N^2), jadi butuh waktu sedikit)...")
labels = dbscan(X, eps=EPS_KM, min_pts=MIN_PTS)

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = np.sum(labels == -1)
print(f"Cluster ditemukan: {n_clusters}")
print(f"Titik noise: {n_noise} / {labels.size}")

# Untuk visualisasi: gabung label dengan posisi penuh?
# Kita visualisasikan hanya subset X (yang di-DBSCAN).
xs = X[:, 0]
ys = X[:, 1]
zs = X[:, 2]

# -----------------------------
# 6. Buat bola bumi (sphere)
# -----------------------------
phi = np.linspace(0, np.pi, 60)        # polar angle
theta = np.linspace(0, 2 * np.pi, 60)  # azimuth
phi, theta = np.meshgrid(phi, theta)

x_earth = R_EARTH * np.sin(phi) * np.cos(theta)
y_earth = R_EARTH * np.sin(phi) * np.sin(theta)
z_earth = R_EARTH * np.cos(phi)

earth_surface = go.Surface(
    x=x_earth,
    y=y_earth,
    z=z_earth,
    colorscale="Blues",
    showscale=False,
    opacity=0.8,
    name="Earth",
)

# -----------------------------
# 7. Scatter 3D satelit dengan warna cluster
# -----------------------------
# Agar -1 (noise) terlihat beda, kita shift label
labels_for_color = labels.astype(float)
labels_for_color[labels_for_color == -1] = -0.5  # noise

sat_scatter = go.Scatter3d(
    x=xs,
    y=ys,
    z=zs,
    mode="markers",
    marker=dict(
        size=2,
        opacity=0.9,
        color=labels_for_color,
        colorscale="Turbo",
        colorbar=dict(title="Cluster ID"),
    ),
    text=names_sub,
    hovertemplate="Cluster %{marker.color}<br>%{text}<br>x=%{x:.0f} km<br>y=%{y:.0f} km<br>z=%{z:.0f} km",
    name="Satellites",
)

fig = go.Figure(data=[earth_surface, sat_scatter])

fig.update_layout(
    title=(
        "All Active Satellites (subset) clustered with DBSCAN<br>"
        f"eps={EPS_KM} km, min_pts={MIN_PTS}, clusters={n_clusters}, noise={n_noise}"
    ),
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode="data",
    ),
    margin=dict(l=0, r=0, t=60, b=0),
)

fig.show()


Mengambil data satelit dari CelesTrak...
Jumlah entri satelit (raw): 13499
Jumlah satelit dengan posisi valid: 13499
DBSCAN hanya akan dijalankan pada subset 3000 satelit.
Menjalankan DBSCAN (ini O(N^2), jadi butuh waktu sedikit)...
Cluster ditemukan: 1
Titik noise: 268 / 3000
